In [45]:
import theano
import numpy as np

In [2]:
import theano.tensor as T

In [7]:
x = T.scalar()
y = T.scalar()

z = x + y

f = theano.function(inputs=[x,y], outputs=z)

In [8]:
f(2,3)

array(5.0)

In [9]:
theano.printing.debugprint(f)

Elemwise{add,no_inplace} [id A] ''   0
 |<TensorType(float64, scalar)> [id B]
 |<TensorType(float64, scalar)> [id C]


In [11]:
theano.pp(z)

'(<TensorType(float64, scalar)> + <TensorType(float64, scalar)>)'

In [13]:
x = T.scalar('x', dtype='int32')
y = 2 * x
f = theano.function(inputs=[x], outputs=y)

In [14]:
theano.printing.debugprint(f)

Elemwise{mul,no_inplace} [id A] ''   0
 |TensorConstant{2} [id B]
 |x [id C]


In [15]:
x = T.scalar('x', dtype='int32')
y = 2 * x
f = theano.function(inputs=[x], outputs=[y,y**2])

In [16]:
f(3)

[array(6, dtype=int32), array(36, dtype=int32)]

In [17]:
#zadanien obliczy 2x - y - 2x. wypisz graf obliczen
x = T.scalar('x', dtype='int32')
y = T.scalar('y', dtype='int32')
z = 2 * x - y - 2 * x
f = theano.function(inputs=[x,y], outputs=z)

In [18]:
theano.printing.debugprint(f)

Elemwise{neg,no_inplace} [id A] ''   0
 |y [id B]


In [19]:
# optymalizacja grafu obliczen. 2*x - y - 2*x zostalo skrocone do -y

In [20]:
u = T.vector()
v = T.vector()
m = T.matrix()

w = u + v

y = T.dot(m,w)

f = theano.function(inputs=[u,v,m], outputs=y)
theano.printing.debugprint(f)

CGemv{inplace} [id A] ''   3
 |AllocEmpty{dtype='float64'} [id B] ''   2
 | |Shape_i{0} [id C] ''   1
 |   |<TensorType(float64, matrix)> [id D]
 |TensorConstant{1.0} [id E]
 |<TensorType(float64, matrix)> [id D]
 |Elemwise{add,no_inplace} [id F] ''   0
 | |<TensorType(float64, vector)> [id G]
 | |<TensorType(float64, vector)> [id H]
 |TensorConstant{0.0} [id I]


In [22]:
a = theano.shared(7)
a

<TensorType(int64, scalar)>

In [23]:
a.get_value()

array(7)

In [25]:
a.set_value(4)
a.get_value()

array(4)

In [26]:
a = 3
a

3

In [28]:
licznik = theano.shared(0)
x = T.scalar()
f = theano.function(inputs=[x], outputs=x*2, updates=[(licznik, licznik+1)])

for i in range(20):
    if i % 7 == 0:
        print(f(i))

0.0
14.0
28.0


In [29]:
licznik.get_value()

array(3)

In [30]:
# obliczanie pochodnych

In [31]:
x = T.scalar()
y = x**2
g = T.grad(y,x)
theano.pp(g)

'((fill((<TensorType(float64, scalar)> ** TensorConstant{2}), TensorConstant{1.0}) * TensorConstant{2}) * (<TensorType(float64, scalar)> ** (TensorConstant{2} - TensorConstant{1})))'

In [32]:
f = theano.function([x],g)

In [33]:
theano.pp(f.maker.fgraph.outputs[0])

'(TensorConstant{2.0} * <TensorType(float64, scalar)>)'

In [37]:
x = T.scalar('x')
y = T.scalar('y')
z = x**2 + y**3
g = T.grad(z,(x,y))

In [39]:
f = theano.function([x,y],g)
theano.pp(f.maker.fgraph.outputs[1])

'Elemwise{Composite{(i0 * sqr(i1))}}(TensorConstant{3.0}, y)'

In [40]:
x = T.scalar('x')
y = T.scalar('y')
z = x**2 + y**3
gx,gy = T.grad(z,(x,y))
theano.pp(gx)

'((fill(((x ** TensorConstant{2}) + (y ** TensorConstant{3})), TensorConstant{1.0}) * TensorConstant{2}) * (x ** (TensorConstant{2} - TensorConstant{1})))'

In [41]:
#petle

In [42]:
def fun(x):
    return 2*x

In [43]:
x = T.vector()
results, _ = theano.scan(fn=fun, sequences=x)

In [46]:
f = theano.function([x], results)
f(np.array([1,2,3]))

array([ 2.,  4.,  6.])

In [47]:
def fun(x, y):
    return x*y

x = T.vector()
y = T.matrix()

res, _ = theano.scan(fn=fun, sequences=[x,y])
f = theano.function([x,y], res)

In [48]:
f([2,3], np.array([[1,2],[3,4]]))

array([[  2.,   4.],
       [  9.,  12.]])

In [50]:
def fun(x):
    return x*x

res, _ = theano.scan(fn=fun, outputs_info=2.0, n_steps=5)
f = theano.function([], res)
f()

array([  4.00000000e+00,   1.60000000e+01,   2.56000000e+02,
         6.55360000e+04,   4.29496730e+09], dtype=float32)

In [51]:
def fun(x):
    return x*x

res, _ = theano.scan(fn=fun, outputs_info=T.cast(2,'int32'), n_steps=5)
f = theano.function([], res)
f()

array([    4,    16,   256, 65536,     0], dtype=int32)

In [52]:
#x[t] = x[t-1] + v[t]

In [60]:
def fun(v_t, x_tm1):
    return x_tm1 + v_t

v = T.vector(dtype='float32')
x, _ = theano.scan(fn = fun, sequences=[v], outputs_info=2.0)
f = theano.function([v], x)

f([1,2,3])

array([ 3.,  5.,  8.], dtype=float32)

In [61]:
#x{t] = y[t-1] + u[t]
#y[t] = x[t-1] + v[t]
#z[t] = x[t] + y[t]

In [67]:
u = T.vector()
v= T.vector()

def fun(u_t, v_t, x_tm1, y_tm1):
    x_t = y_tm1 + u_t
    y_t = x_tm1 + v_t
    return (x_t, y_t, x_t+y_t)

(x,y,z), _ = theano.scan(fn=fun, sequences=[u,v], outputs_info=[theano.shared(2.0),theano.shared(3.0),None])
f = theano.function([u,v], [x,y,z])
f([1,2,3],[1,4,7])

[array([  4.,   5.,  11.]),
 array([  3.,   8.,  12.]),
 array([  7.,  13.,  23.])]

In [68]:
a = T.scalar()
v = T.vector()

def fun(x,p):
    return x**p

res, _ = theano.scan(fn=fun, sequences=[v], non_sequences=[a])

f = theano.function([v,a], outputs=res)

f([1,2,3],2)

array([ 1.,  4.,  9.])

In [69]:
#x[t] = x[t-1] + v[t] + p

In [70]:
a = T.scalar()
v = T.vector()

def fun(v_t, x_tm1, p):
    return x_tm1 + v_t + p

res, _ = theano.scan(fn=fun, sequences=[v], outputs_info=theano.shared(3.0), non_sequences=[a])

f = theano.function([v,a], outputs=res)

f([1,2,3],2)

array([  6.,  10.,  15.])

In [74]:
x= T.vector()
skumulowana_suma = theano.shared(0.0)

def fun(x):
    return 2*x, {skumulowana_suma: skumulowana_suma+x}

res, upd = theano.scan(fn=fun, sequences=[x])

f = theano.function([x], outputs=res, updates=upd)
print(f([1,2,3]))
skumulowana_suma.get_value()

[ 2.  4.  6.]


array(6.0)

In [ ]:
#theano.ifelse.ifelse